<a href="https://colab.research.google.com/github/nxxk23/AI-Engineer/blob/main/fraud/insurance_json.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install langchain langchain_huggingface langchain_community  gradio transformers elasticsearch gradio_calendar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 51.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 21.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 543.1/543.1 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 20.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 90.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 69.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 

In [6]:
import os
from getpass import getpass
import requests
import os
from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from huggingface_hub import InferenceClient
from elasticsearch import Elasticsearch
import pandas as pd
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F
from sentence_transformers import SentenceTransformer
import json
import gradio as gr
from datetime import datetime

## gradio

In [7]:
def handle_form(data_json):
    try:
        data = json.loads(data_json)

        policy_start_date = data.get("policy_start_date")
        policy_end_date = data.get("policy_end_date")
        visit_date = data.get("visit_date")
        diagnosis = data.get("diagnosis", "")

        if not all([policy_start_date, policy_end_date, visit_date]):
            raise ValueError("Missing required date fields in input JSON.")

        policy_start = datetime.strptime(policy_start_date, '%Y-%m-%d')
        policy_end = datetime.strptime(policy_end_date, '%Y-%m-%d')
        visit = datetime.strptime(visit_date, '%Y-%m-%d')
        icd10_code = diagnosis.split(' ')[0] if diagnosis else None

        wait_disease = ["K30", "K297", "I495"]

        if icd10_code in wait_disease:
            result = {
                "text": "Decline ❌",
                "entities": [{"start": 0, "end": 7, "label": "Decline"}]
            }
            reason = f'เนื่องจาก {diagnosis} เป็นโรคที่มีระยะเวลาการรอคอย 180 วัน'

        elif visit < policy_start or visit > policy_end:
            result = {
                "text": "Decline ❌",
                "entities": [{"start": 0, "end": 7, "label": "Decline"}]
            }
            reason = f"Visit date {visit_date} is outside the policy coverage period ({policy_start_date} to {policy_end_date})."
        else:
            result = {
                "text": "Approve ✅",
                "entities": [{"start": 0, "end": 7, "label": "Approve"}]
            }
            reason = "Visit date is within the policy coverage period."

        return result, reason

    except ValueError as e:
        result = {
            "text": "Error ⚠️",
            "entities": [{"start": 0, "end": 6, "label": "Error"}]
        }
        reason = f"Invalid input or date format. Please use YYYY-MM-DD. {str(e)}"
        return result, reason

In [10]:
custom_css = """
.red-border {
    border: 2px solid red !important;
    border-radius: 5px;
    padding: 5px;
}
"""

### Ui
with gr.Blocks(css=custom_css) as demo:
    gr.Markdown("## SmartClaim ฝั่งประกัน 🛡️")
    data = gr.Textbox(label="ข้อมูลผู้ป่วย", placeholder="กรอกข้อมูลผู้ป่วยในรูปแบบ JSON แล้ว Enter หรือ Submit")
    submit_button = gr.Button("Submit")
    gr.Markdown("### Insurance Claim Decision System")
    with gr.Row(elem_classes="red-border"):
        result_output = gr.HighlightedText(label="Result", interactive=False)
        reason_output = gr.Textbox(label="Reason", interactive=False)

        data.submit(
            fn=handle_form,
            inputs=[data],
            outputs=[result_output, reason_output]
        )
        submit_button.click(
            fn=handle_form,
            inputs=[data],
            outputs=[result_output, reason_output]
        )

In [11]:
demo.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5daef2eb3c7ca786db.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5daef2eb3c7ca786db.gradio.live


## **decline case**

In [ ]:
##  ไม่ครอบคลุมจาก Uncoverage Period
##  policy_start_date, policy_end_date, visit_date
{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "คณิศร",
    "last_name": "ทองดีบุตร",
    "patient_id": "6609393",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-13",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2023-01-01",
    "policy_end_date": "2024-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "J069 การติดเชื้อทางเดินหายใจส่วนบนเฉียบพลัน ไม่ระบุรายละเอียด (Acute upper respiratory infection, unspecified [ URI ])",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "7000",
    "total_net": "1491.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}

In [ ]:
##  โรคที่มีระยะเวลารอคอย 180 วัน
##  icd_10 from diagnosis  ["K30", "K297", "I495"]

{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "aaa",
    "last_name": "bbb",
    "patient_id": "147852369",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-13",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2024-01-01",
    "policy_end_date": "2025-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "K30 อาหารไม่ย่อย (Functional Dyspepsia)",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "7000",
    "total_net": "1491.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}

{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "aaa",
    "last_name": "bbb",
    "patient_id": "147852369",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-13",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2024-01-01",
    "policy_end_date": "2025-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "K297 กระเพาะอาหารอักเสบ ไม่ระบุรายละเอียด (Gastritis, unspecified)",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "7000",
    "total_net": "1491.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}

{
    "id_card": "1309902530511",
    "title": "นาย",
    "first_name": "aaa",
    "last_name": "bbb",
    "patient_id": "147852369",
    "birth_date": "1997-12-22",
    "visit_date": "2024-08-13",
    "treatment_type": "เจ็บป่วย",
    "visit_type": "เข้ารับการรักษาครั้งแรก",
    "policy_start_date": "2024-01-01",
    "policy_end_date": "2025-01-01",
    "chief_complaint": "ปวดศีรษะ เจ็บคอSpo2=97%",
    "diagnosis": "I495 กลุ่มอาการซิคไซนัส (Sick sinus syndrome)",
    "billings": "[{'billing_initial': '712.00', 'billing_discount': '71.00', 'simb_billing_code': '1.1.1(3)', 'billing_net_amount': '641.00', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'billing_initial': '100', 'billing_discount': '0', 'simb_billing_code': '1.1.12', 'billing_net_amount': '100.00', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'billing_initial': '150', 'billing_discount': '0', 'simb_billing_code': '1.1.14(2)', 'billing_net_amount': '150.00', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'billing_initial': '600', 'billing_discount': '0', 'simb_billing_code': '1.2.1', 'billing_net_amount': '600.00', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "order_items": "[{'initial': '50', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Ambroxol TAB 30  mg', 'net_amount': '50.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '300', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Claron TAB 500 mg ;', 'net_amount': '300.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Exib (Etoricoxib) TA', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '42', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Loratadine (Carinose', 'net_amount': '42.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '180', 'item_id': 'DRUG', 'discount': '71', 'item_name': 'Rotuss TAB 10/100 mg', 'net_amount': '109.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '40', 'item_id': 'DRUG', 'discount': '0', 'item_name': 'Tylenol TAB 500 mg', 'net_amount': '40.00', 'item_amount': '1', 'simb_billing_code': '1.1.1(3)', 'local_billing_code': 'DRUG', 'local_billing_name': 'Outpatient Medication   ค่ายาผู้ป่วยนอก'}, {'initial': '100', 'item_id': 'SER', 'discount': '0', 'item_name': 'ค่าบริการพยาบาล', 'net_amount': '100.00', 'item_amount': '1', 'simb_billing_code': '1.1.12', 'local_billing_code': 'SER', 'local_billing_name': 'Nursing and Midwifery Charge   ค่าบริการพยาบาล'}, {'initial': '150', 'item_id': 'SER2', 'discount': '0', 'item_name': 'ค่าบริการโรงพยาบาลผู', 'net_amount': '150.00', 'item_amount': '1', 'simb_billing_code': '1.1.14(2)', 'local_billing_code': 'SER2', 'local_billing_name': 'Other Medical Service Charge ค่าบริการทางการแพทย์'}, {'initial': '600', 'item_id': 'DFO', 'discount': '0', 'item_name': 'ค่าตรวจรักษาทั่วไป', 'net_amount': '600.00', 'item_amount': '1', 'simb_billing_code': '1.2.1', 'local_billing_code': 'DFO', 'local_billing_name': 'Physician Evaluation and Management Services'}]",
    "remaining_opd_year": "7000",
    "total_net": "1491.0",
    "final_result": "6657.75",
    "recommendation_adjustment": "+ 5166.75"
}
